In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [2]:
def load_and_preprocess_images(folder_path, image_size=(64, 64)):
    image_data = []
    labels = []

    for label_idx, label_name in enumerate(os.listdir(folder_path)):
        label_folder = os.path.join(folder_path, label_name)
        
        for image_filename in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_filename)
            image = Image.open(image_path).convert('L')  # Convert to grayscale
            image = image.resize(image_size)
            image = np.array(image) / 255.0  # Normalize pixel values
            image_data.append(image)
            labels.append(label_idx)

    return np.array(image_data), np.array(labels)

In [3]:
dataset_folder = "Dataset"

In [4]:
X, y = load_and_preprocess_images(dataset_folder)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
input_size = X_train.shape[1] * X_train.shape[2]
hidden_size = 64
output_size = 3
learning_rate = 0.01
num_epochs = 1000

In [7]:
np.random.seed(42)
weights_input_hidden = np.random.randn(input_size, hidden_size)
bias_hidden = np.zeros((1, hidden_size))
weights_hidden_output = np.random.randn(hidden_size, output_size)
bias_output = np.zeros((1, output_size))

In [8]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [9]:
for epoch in range(num_epochs):
    # Forward pass
    hidden_layer_input = np.dot(X_train.reshape(-1, input_size), weights_input_hidden) + bias_hidden
    hidden_layer_output = sigmoid(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
    output_layer_output = sigmoid(output_layer_input)

    # Compute loss (cross-entropy)
    num_examples = len(X_train)
    y_encoded = np.zeros((num_examples, output_size))
    y_encoded[range(num_examples), y_train] = 1
    loss = -np.sum(y_encoded * np.log(output_layer_output + 1e-9)) / num_examples

    # Backpropagation
    d_loss = (output_layer_output - y_encoded) / num_examples
    d_output_layer = d_loss * output_layer_output * (1 - output_layer_output)
    d_hidden_layer = np.dot(d_output_layer, weights_hidden_output.T) * hidden_layer_output * (1 - hidden_layer_output)

    # Update weights and biases
    weights_hidden_output -= learning_rate * np.dot(hidden_layer_output.T, d_output_layer)
    bias_output -= learning_rate * np.sum(d_output_layer, axis=0, keepdims=True)
    weights_input_hidden -= learning_rate * np.dot(X_train.reshape(-1, input_size).T, d_hidden_layer)
    bias_hidden -= learning_rate * np.sum(d_hidden_layer, axis=0, keepdims=True)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


Epoch 0, Loss: 2.5599
Epoch 100, Loss: 2.5704
Epoch 200, Loss: 2.5824
Epoch 300, Loss: 2.5921
Epoch 400, Loss: 2.6014
Epoch 500, Loss: 2.6116
Epoch 600, Loss: 2.6243
Epoch 700, Loss: 2.6389
Epoch 800, Loss: 2.6539
Epoch 900, Loss: 2.6680


In [10]:
hidden_layer_input = np.dot(X_test.reshape(-1, input_size), weights_input_hidden) + bias_hidden
hidden_layer_output = sigmoid(hidden_layer_input)
output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
output_layer_output = sigmoid(output_layer_input)


In [11]:
predictions = np.argmax(output_layer_output, axis=1)


In [12]:
accuracy = accuracy_score(y_test, predictions)
f1_m = f1_score(y_test, predictions, average='macro')
f1_w = f1_score(y_test, predictions, average='weighted')

In [13]:
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Macro F1 Score: {f1_m:.4f}")
print(f"Weighted F1 Score: {f1_w:.4f}")

Test Accuracy: 49.68%
Macro F1 Score: 0.2768
Weighted F1 Score: 0.4439
